In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
import datetime

from tqdm import tqdm
import time
import sys
import glob

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['../features/all_features/173_l35_org_lag3_5_auth1_monthly_avg_purchase_amount_min@.gz', '../features/all_features/173_l35_org_lag3_5_auth1_monthly_avg_purchase_amount_sum@.gz', '../features/all_features/173_l35_org_lag3_5_auth1_monthly_avg_month_lag_std@.gz', '../features/all_features/173_l35_org_lag3_5_auth1_monthly_avg_category_2_1_0_mean@.gz', '../features/all_features/173_l35_org_lag3_5_auth1_monthly_avg_yyyy_week_nunique@.gz']


In [34]:
hist_lag = 68
new_lag = 910

hist_path_list = glob.glob(f'../features/all_features/*l{hist_lag}*auth1*')
hist_path_list = [p for p in hist_path_list if not(p.count('new'))]
month_hist_path_list = [p for p in hist_path_list if p.count('monthly')]

new_path_list = glob.glob(f'../features/all_features/*l{new_lag}*auth1*new*')
month_new_path_list = [p for p in new_path_list if p.count('monthly')]

prefix = f'202_l{new_lag}'
for month_hist_path in month_hist_path_list:
    month_hist_name = re.search(r'monthly_([^/.]*).gz', month_hist_path).group(1).replace('@', '')
    for month_new_path in month_new_path_list:
        month_new_name = re.search(r'monthly_([^/.]*).gz', month_new_path).group(1).replace('@', '')
        if month_hist_name == month_new_name:
            
            hist_feat = utils.read_pkl_gzip(month_hist_path)
            new_feat = utils.read_pkl_gzip(month_new_path)
            ratio = hist_feat / (new_feat + 1.0e-5)
            diff  = hist_feat - new_feat
            
            utils.to_pkl_gzip(obj=ratio , path=f'../features/1_first_valid/{prefix}_hist_new_ratio_{month_hist_name}')
            utils.to_pkl_gzip(obj=diff , path=f'../features/1_first_valid/{prefix}_hist_new_diff_{month_hist_name}')